# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gts
import holoviews as hv
import matplotlib.pyplot as plt
hv.extension('matplotlib')
hv.extension('bokeh')


%matplotlib inline




# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
output_path = r"C:\Users\lenar\OneDrive\Documents\Data Bootcamp\python-api-challenge\WeatherPy\Starter_Code\Starter_Code\output_data\cities.csv"
city_data_df = pd.read_csv(output_path)

# Convert Kelvin to Celsius
city_data_df['Max Temp (C)'] = city_data_df['Max Temp'] - 273.15

# Display sample data with temperature in Celsius, keeping 'Wind Speed' and 'Cloudiness'
print(city_data_df[['City', 'Max Temp', 'Max Temp (C)', 'Wind Speed', 'Cloudiness']].head(10))


                City  Max Temp  Max Temp (C)  Wind Speed  Cloudiness
0               mana    299.23         26.08        3.79          85
1  port-aux-francais    278.88          5.73       10.77          99
2            bartica    300.33         27.18        1.81          88
3               nuuk    274.49          1.34       18.01         100
4            stanley    272.33         -0.82        0.51          40
5          grytviken    274.94          1.79        4.91          28
6          dhidhdhoo    301.01         27.86        0.31          93
7              crane    289.30         16.15        2.86          11
8             bethel    277.70          4.55        7.20         100
9           belmonte    299.31         26.16        5.52          83


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%matplotlib inline

# Create a GeoViews Dataset
gdf = gv.Dataset(city_data_df, kdims=['Lng', 'Lat'], vdims=['City', 'Humidity'])

# Create a GeoViews Points element with Humidity as the size
scatter_plot = gdf.to(gv.Points, kdims=['Lng', 'Lat'], vdims=['City', 'Humidity']).opts(
size=hv.dim('Humidity') * 0.07, 
color='City',
) 


# Create a base map
tiles = gts.CartoLight

# Overlay the points on the base map
city_map = tiles * scatter_plot

# Create a separate legend
legend = hv.NdOverlay({city: hv.Points([0, 0], label=city).opts(color=color)
                      for city, color in zip(city_data_df['City'], scatter_plot.opts['color'])})

# Convert GeoViews plot to Matplotlib plot
#mpl_plot = gv.render(city_map, backend='matplotlib')
city_map.opts(width=800, height=500)
# Display the Matplotlib plot
#mpl_plot = gv.render(city_map, backend='matplotlib')
#plt.show()

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Convert Kelvin to Celsius
city_data_df['Max Temp (C)'] = city_data_df['Max Temp'] - 273.15

# Adjusted criteria for broader temperature range
adjusted_weather_df = city_data_df[
    (city_data_df['Max Temp (C)'] < 27) & (city_data_df['Max Temp (C)'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
adjusted_weather_df = adjusted_weather_df.dropna()

# Display sample data
print(adjusted_weather_df.head())


     City_ID           City      Lat       Lng  Max Temp  Humidity  \
69        69      carnarvon -24.8667  113.6333    299.32        31   
76        76        banfora  10.6333   -4.7667    297.70        14   
303      303        salalah  17.0151   54.0924    295.20        28   
350      350  yung shue wan  22.2333  114.1167    296.86        86   
379      379       vallenar -28.5708  -70.7581    296.28        43   

     Cloudiness  Wind Speed Country        Date  Max Temp (C)  
69            0        2.88      AU  1702337296         26.17  
76            0        1.98      BF  1702337139         24.55  
303           0        3.09      OM  1702337425         22.05  
350           0        2.21      HK  1702337090         23.71  
379           0        1.49      CL  1702337468         23.13  


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Assuming 'adjusted_weather_df' is your DataFrame with adjusted weather conditions
hotel_df = adjusted_weather_df[['City', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column for "Hotel Name"
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())


              City      Lat       Lng  Humidity Hotel Name
69       carnarvon -24.8667  113.6333        31           
76         banfora  10.6333   -4.7667        14           
303        salalah  17.0151   54.0924        28           
350  yung shue wan  22.2333  114.1167        86           
379       vallenar -28.5708  -70.7581        43           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000  # Assuming you want to search within a 10,000 meters radius
params = {
    'categories': 'hotel',
    'radius': radius,
    'limit': 1,  # Limit to the first result
    'apiKey': geoapify_key  # Replace with your actual Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lon = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point:{lon},{lat}"
    params["bias"] = f"circle:{radius}@{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
carnarvon - nearest hotel: No hotel found
banfora - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
yung shue wan - nearest hotel: No hotel found
vallenar - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
ziguinchor - nearest hotel: No hotel found
maceio - nearest hotel: No hotel found
chandler - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Hotel Name
69,carnarvon,-24.8667,113.6333,31,No hotel found
76,banfora,10.6333,-4.7667,14,No hotel found
303,salalah,17.0151,54.0924,28,No hotel found
350,yung shue wan,22.2333,114.1167,86,No hotel found
379,vallenar,-28.5708,-70.7581,43,No hotel found
459,karratha,-20.7377,116.8463,44,No hotel found
488,ziguinchor,12.5833,-16.2719,52,No hotel found
510,maceio,-9.6658,-35.7353,94,No hotel found
567,chandler,33.3062,-111.8412,18,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
import geoviews as gv
import geoviews.tile_sources as gts

# Define a GeoViews element using Points
cities_element = gv.Points(
    hotel_df, 
    kdims=['Lng', 'Lat'], 
    vdims=['City', 'Humidity', 'Hotel Name'],
    label='City Locations'
).opts(
    size=10, 
    tools=['hover'], 
    color='pink',
    hover_color='pink',
    hover_fill_color='pink',
    hover_line_color='black',  
    width=800,  # Set the width of the plot
    height=500,  # Set the height of the plot
)

# Create a basemap using GeoViews tile_sources
basemap = gts.EsriImagery()

# Combine the basemap and the GeoViews element
geo_plot = basemap * cities_element

# Display the GeoViews plot
geo_plot


:Overlay
   .WMTS.I                :WMTS   [Longitude,Latitude]
   .Points.City_Locations :Points   [Lng,Lat]   (City,Humidity,Hotel Name)